In [1]:
import pandas as pd

df_nirvana = pd.read_csv('Dados/nirvana_lyrics.csv')
df_pink_floyd = pd.read_csv('Dados\pink_floyd_lyrics.csv')

In [14]:
import re
def arruma(lyrics):
    if isinstance(lyrics, str):
        lyrics = lyrics.lower()
        lyrics = re.sub(r'[.!@#$%&*(){}:;_?/<>+|=,\[\]\n]', '', lyrics)

    return lyrics

df_nirvana['lyrics'] = df_nirvana['lyrics'].astype(str)
df_pink_floyd['lyrics'] = df_pink_floyd['lyrics'].astype(str)

df_nirvana['lyrics'] = df_nirvana['lyrics'].apply(arruma)
df_pink_floyd['lyrics'] = df_pink_floyd['lyrics'].apply(arruma)

In [18]:
def calcula(lyrics, palavra_escolhida):
    palavras = lyrics.split()
    contador = palavras.count(palavra_escolhida)
    todas_palavras = len(palavras)
    probabilidade = contador / todas_palavras
    return probabilidade


probabilidade_nirvana = calcula(" ".join(df_nirvana['lyrics']), "your")
probabilidade_pink_floyd = calcula(" ".join(df_pink_floyd['lyrics']), "your")

print(f'Probabilidade da palavra "YOUR" aparecer no Nirvana: {probabilidade_nirvana:.4f}')
print(f'Probabilidade da palavra "YOUR" aparecer no Pink Floyd: {probabilidade_pink_floyd:.4f}')

Probabilidade da palavra "YOUR" aparecer no Nirvana: 0.0070
Probabilidade da palavra "YOUR" aparecer no Pink Floyd: 0.0080
